In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.converters import (InequalityToEquality, IntegerToBinary, 
                                            LinearEqualityToPenalty, LinearInequalityToPenalty,
                                            MaximizeToMinimize)
from docplex.mp.model import Model

# SciPy minimizer routine
from scipy.optimize import minimize

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session
from qiskit.result import QuasiDistribution
from qiskit.circuit.library import QAOAAnsatz

In [2]:
service = QiskitRuntimeService()

In [3]:
class QuadraticProgram2Ising:
    """A mapping of `QuadraticProgram` to a Ising Hamiltonian
    in `SparsePauliOp` format
    """
    def __init__(self):
        self.input_types = (QuadraticProgram, )
        self.output_types = (SparsePauliOp, )

    def run(self, program, ansatz = None):
        if not isinstance(program, self.input_types):
            raise TypeError(f"Program is invalid input type, must be one of {self.input_types}")
        ising, constant = program.to_ising(opflow=False)
        return ising + SparsePauliOp.from_list([("I"*ising.num_qubits, constant)])

In [4]:
class QAOASolver:
    def __init__(self, session, qaoa_reps=2):
        self.input_types = (SparsePauliOp, )
        self.output_types = (QuasiDistribution, )
        self.session = session
        self.qaoa_reps = 2

    def run(self, hamiltonian):
        ansatz = QAOAAnsatz(hamiltonian, reps=2)
        estimator = Estimator(session=session, options={"shots": int(1e4)})
        sampler = Sampler(session=session, options={"shots": int(1e4)})

        def cost_func(params, ansatz, hamiltonian, estimator):
            """Return estimate of energy from estimator
        
            Parameters:
                params (ndarray): Array of ansatz parameters
                ansatz (QuantumCircuit): Parameterized ansatz circuit
                hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
                estimator (Estimator): Estimator primitive instance
        
            Returns:
                float: Energy estimate
            """
            cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
            return cost

        x0 = 2 * np.pi * np.random.rand(ansatz.num_parameters)
        res = minimize(cost_func, x0, args=(ansatz, hamiltonian, estimator), method="COBYLA")
        # Assign solution parameters to ansatz
        qc = ansatz.assign_parameters(res.x)
        # Add measurements to our circuit
        qc.measure_all()
        # Sample ansatz at optimal parameters
        samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
        # Close the session since we are now done with it
        session.close()
        return samp_dist

In [5]:
import copy
import warnings

class PropertySet(dict):
    """A default dictionary-like object"""
    def __missing__(self, key):
        return None

class CompositeWorkflow:
    def __init__(self, passes, name=None, store_final_output=False, strict_validation=True):
        self.passes = passes
        self.name = name
        self.store_final_output = store_final_output
        self.input_types = None
        self.output_types = None
        self.strict_validation = strict_validation
        self._validate_passes()
        self.property_set = PropertySet()

    def _validate_passes(self):
        self.input_types = self.passes[0].input_types
        input_types = self.passes[0].input_types
        for idx, individual_pass in enumerate(self.passes):
            # If the pass is itself a CompositeWorkflow then run its validation
            if isinstance(individual_pass, CompositeWorkflow):
                individual_pass._validate_passes()
                continue
            temp_input_types = individual_pass.input_types
            # Look to see if there is overlap between last passes outputs
            # and this passes inputs
            input_overlap = set(input_types).intersection(set(temp_input_types))
            # If no overlap, bad news
            if not input_overlap:
                raise Exception(f"{temp_input_types} not compatible with {input_types}")
            # If partial overlap, then can only weakly validate the workflow, raise
            # unless explicitly disabled
            if len(input_overlap) != len(temp_input_types) and self.strict_validation:
                diffs = set(input_types).difference(set(temp_input_types))
                raise Exception(f"Possible inputs {diffs} not valid for this pass")
            input_types = individual_pass.output_types
        self.output_types = individual_pass.output_types
    
    def run(self, input):
        temp = copy.deepcopy(input)
        working_props = self.property_set
        for individual_pass in self.passes:
            individual_pass.property_set = working_props
            temp = individual_pass.run(temp)
            if isinstance(individual_pass, CompositeWorkflow):
                if individual_pass.store_final_output:
                    working_props[individual_pass.name] = {"final_output": temp}
        return temp

In [12]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x - 2*y + 3*z + x*y + x*z + 2*y*z)
#print(mdl.export_as_lp_string())

In [14]:
backend = service.get_backend('ibmq_qasm_simulator')

In [15]:
session = Session(backend=backend)

In [13]:
qp = QuadraticProgram.from_docplex_mp(mdl)

In [25]:
transform = CompositeWorkflow([InequalityToEquality(), # Transformation
                        IntegerToBinary(), # Transformation
                        LinearEqualityToPenalty(), # Transformation
                        LinearInequalityToPenalty(), # Transformation
                        MaximizeToMinimize(), # Transformation
                        ], name='qubo-transformer', store_final_output=True)

In [26]:
cw = CompositeWorkflow([transform,
                        QuadraticProgram2Ising(), # Quantum part - to Hamiltonian
                        QAOASolver(session), # Quantum part - execution on QPU
                        EvaluateProgramSolution() # Quantum part - bit-string to program variables
                        ])

In [27]:
cw.input_types

(qiskit_optimization.problems.quadratic_program.QuadraticProgram,)

In [28]:
cw.output_types

(tuple,)

In [ ]:
ans = cw.run(qp)

Traceback (most recent call last):
  File "/Users/paul/mambaforge/envs/qiskit-310/lib/python3.10/site-packages/scipy/optimize/_cobyla_py.py", line 273, in calcfc
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = fun(np.copy(x), *args)
  File "/var/folders/01/dp6k619n4dng8l43hj8z2nl80000gn/T/ipykernel_78547/3649083624.py", line 25, in cost_func
  File "/Users/paul/mambaforge/envs/qiskit-310/lib/python3.10/site-packages/qiskit_ibm_runtime/estimator.py", line 147, in run
    return super().run(
  File "/Users/paul/mambaforge/envs/qiskit-310/lib/python3.10/site-packages/qiskit/primitives/base/base_estimator.py", line 188, in run
    return self._run(
  File "/Users/paul/mambaforge/envs/qiskit-310/lib/python3.10/site-packages/qiskit_ibm_runtime/estimator.py", line 185, in _run
    return self._run_primitive(
  File "/U

In [23]:
ans

(-2.0, [('x', '0'), ('y', '1'), ('z', '0')])

In [10]:
def evaluate_quadratic_program(bitstring, program):
    constant = program.objective.constant
    linear_elements = program.objective.linear.to_dict()
    quadratic_elements = program.objective._quadratic.to_dict()

    # Flip string so 0th bit is 0th array element for easy math
    x = np.fromiter(list(bitstring[::-1]), dtype=int)
    
    sum = constant
    for element, val in linear_elements.items():
        sum += x[element] * val
    for element, val in quadratic_elements.items():
        sum += x[element[0]] * val * x[element[1]]
    return sum

In [24]:
class EvaluateProgramSolution:
    """Evaluate solutions
    """
    def __init__(self, program=None):
        self.program = program
        self.input_types = (QuasiDistribution, )
        self.output_types = (tuple, )
        self.property_set = PropertySet()

    def run(self, dist):
        if self.program is None:
            program = self.property_set['qubo-transformer']['final_output']
        best_val = np.inf
        best_bits = None
        for bitstring in dist.binary_probabilities():
            temp = evaluate_quadratic_program(bitstring, program)
            if temp < best_val:
                best_val = temp
                best_bits = bitstring
        best_bits = best_bits[::-1]
        out = (best_val, [(var.name, int(best_bits[idx])) for idx, var in enumerate(program.variables)])
        if not isinstance(out, self.output_types):
            raise TypeError("Return type not in 'self.output_types'")
        return out